In [1]:
import os 
import pandas as pd
import numpy as np
import math

import geopandas as gpd
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

output_notebook()

Loading BokehJS ...

In [2]:
path = r'C:/Users/ShrekTheOger/Documents/GitHub/final-project-final-project-bowen-and-natasia'
#path = r'C:\Users\engel\Documents\GitHub\final-project-final-project-bowen-and-natasia'
#import warnings
#warnings.filterwarnings('ignore')

In [3]:
df = os.path.join(path+'/refined_data', 'df_income_pop.csv')
df = pd.read_csv(df)
#Combining state_id with county_id into same format as in university shape file
l = []
for i in range(len(df['State_id'])):
    if len(str(df['State_id'][i])) == 1:
        if len(str(df['County_id'][i])) == 1:
            a = '0'+ str(df['State_id'][i]) + '00' + str(df['County_id'][i])
        elif len(str(df['County_id'][i])) == 2:
            a = '0'+ str(df['State_id'][i]) + '0' + str(df['County_id'][i])
        else:
            a = '0'+ str(df['State_id'][i]) + str(df['County_id'][i])
    else:
        if len(str(df['County_id'][i])) == 1:
            a = str(df['State_id'][i]) + '00' + str(df['County_id'][i])
        elif len(str(df['County_id'][i])) == 2:
            a = str(df['State_id'][i]) + '0' + str(df['County_id'][i])
        else:
            a = str(df['State_id'][i]) + str(df['County_id'][i])
    l.append(a)
df.insert(3, 'GEOID', l)
df.drop(labels=['State_id', 'County_id'], axis=1, inplace = True)
df

,Year,State,County,GEOID,COUNTYFIPS,Total_population,Total_native,Total_born_in_state,Total_born_out_state,Total_born_outside_us,Total_foreign_born,Income_past12m
0,2010,California,Merced County,06047,6047,250699,188498,156434,29768,2296,62201,18041.0
1,2010,California,Modoc County,06049,6049,9605,8971,6067,2882,22,634,20536.0
2,2010,California,Mono County,06051,6051,13905,11341,7059,4108,174,2564,27321.0
3,2010,California,Monterey County,06053,6053,407435,284647,208218,69900,6529,122788,25776.0
4,2010,California,Napa County,06055,6055,134051,103891,73787,27759,2345,30160,34310.0
...,...,...,...,...,...,...,...,...,...,...,...,...
31069,2019,Tennessee,Crockett County,47033,47033,14399,13774,11231,2490,53,625,23771.0
31070,2019,Tennessee,Lake County,47095,47095,7401,7307,5436,1839,32,94,15732.0
31071,2019,Tennessee,Knox County,47093,47093,461104,438362,272843,161946,3573,22742,33229.0
31072,2019,Washington,Benton County,53005,53005,197518,176385,104182,70549,1654,21133,32882.0


In [4]:
county_shp = os.path.join(path+'/raw_data', 'cb_2020_us_county_20m', 'cb_2020_us_county_20m.shp')
county = gpd.read_file(county_shp)

In [5]:
df_shape = county.merge(df, how = 'inner', on='GEOID').set_geometry('geometry')


In [6]:
#df_shape.T

In [7]:
def json_data(selectedYear):
    yr = selectedYear
    
    # Pull selected year from state data
    df_yr = df_shape[df_shape['Year'] == yr]
    
    merged_json = json.loads(df_yr.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data

In [8]:
df_shape['Total_foreign_born'].max()

3485724

In [9]:
# This dictionary contains the formatting for the data in the plots
format_data = [('Total_population', 0, 12000000,'0,0', 'Total Population'),
               ('Total_native', 0, 6750000,'0,0', 'Total Native'),
               ('Total_born_in_state', 0, 5200000,'0,0', 'Total Born in State'),
               ('Total_born_out_state', 0, 2000000,'0,0', 'Total Born out State'),
               ('Total_born_outside_us', 0, 130000,'0,0', 'Total Born out State'),
               ('Total_foreign_born', 0, 3500000,'0,0', 'Total Foreigner'),
               ('Income_past12m', 0, 80000,'$0,0', 'Income in Past 12 Months') 
              ]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

In [10]:
format_df

,field,min_range,max_range,format,verbage
0,Total_population,0,12000000,"0,0",Total Population
1,Total_native,0,6750000,"0,0",Total Native
2,Total_born_in_state,0,5200000,"0,0",Total Born in State
3,Total_born_out_state,0,2000000,"0,0",Total Born out State
4,Total_born_outside_us,0,130000,"0,0",Total Born out State
5,Total_foreign_born,0,3500000,"0,0",Total Foreigner
6,Income_past12m,0,80000,"$0,0",Income in Past 12 Months


In [11]:
#format_df.loc[format_df['verbage'] == 'Total Population', 'field'].iloc[0]

In [12]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # The input yr is the year selected from the slider
    yr = slider.value
    new_data = json_data(yr)
    #print(yr)
    
    # The input cr is the criteria selected from the select box
    cr = select.value
    # input_field = cr
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    #layout = column(p, widgetbox(select), widgetbox(slider))
    layout = column(p, select, slider)
    curdoc().clear()
    curdoc().add_root(layout)

    # Update the data
    geosource.geojson = new_data
    
    #output_notebook()
    #show(layout)

In [13]:
# Create a plotting function
def make_plot(field_name):    
  # Set the format of the colorbar
  min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
  max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
  field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

  # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
  color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

  # Create color bar.
  format_tick = NumeralTickFormatter(format=field_format)
  color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, border_line_color=None, location = (0, 0))

  # Create figure object.
  verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

  p = figure(title = verbage, 
             plot_height = 650, plot_width = 850,
             toolbar_location = None)
  p.xgrid.grid_line_color = None
  p.ygrid.grid_line_color = None
  p.axis.visible = False

  # Add patch renderer to figure. 
  p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
  
  # Specify color bar layout.
  p.add_layout(color_bar, 'right')

  # Add the hover tool to the graph
  p.add_tools(hover)
  return p

In [14]:
# Input geojson source that contains features for plotting for:
# initial year 2018 and initial criteria sale_price_median
geosource = GeoJSONDataSource(geojson = json_data(2010))
input_field = 'Income_past12m'

# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Add hover tool
hover = HoverTool(tooltips = [ ('County','@NAME'),
                               ('State', '@State'),
                               ('Total population', '@Total_population{,}'),
                               ('Born instate', '@Total_born_in_state{,}'),
                               ('Born outstate', '@Total_born_out_state{,}'),])

# Call the plotting function
p = make_plot(input_field)

# Make a slider object: slider 
slider = Slider(title = 'Year',start = 2010, end = 2019, step = 1, value = 2015)
#slider.on_change('value', update_plot)

# Make a selection object: select
select = Select(title='Select Criteria:', value='Income_past12m', options=['Income in Past 12 Months', 'Total Population'])
#select.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
# Display the current document
#layout = column(p, widgetbox(select), widgetbox(slider))
layout = column(p, select, slider)
#curdoc().add_root(layout)
#output_notebook()
#show(layout)


def modify_doc(doc):
    doc.add_root(column(layout))
    slider.on_change('value', update_plot)
    select.on_change('value', update_plot)

handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)